networkx-temporal
---

**[PyPI package](https://pypi.org/p/networkx-temporal/)** | **[Documentation](https://networkx-temporal.readthedocs.io/en/latest)** | **[GitHub repository](https://github.com/nelsonaloysio/networkx-temporal)**


In [ ]:
!pip install -q leidenalg matplotlib networkx networkx-temporal python-igraph

In [ ]:
# %load_ext autoreload
# %autoreload 2
import networkx as nx
import networkx_temporal as tx

___

# Basic operations

The examples below cover the package's basic functionalities, including
how to build a temporal graph, slice it into snapshots, and save and
load the resulting objects to disk.

## Build temporal graph

The main class of the package is the [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph)
object, which extends [NetworkX's
graphs](https://networkx.org/documentation/stable/reference/classes/index.html)
to handle temporal data. Let's start by creating a simple directed graph
using `time` as attribute key:

In [ ]:
import networkx_temporal as tx

TG = tx.TemporalGraph(directed=True, multigraph=False)

TG.add_edge("a", "b", time=0)
TG.add_edge("c", "b", time=1)
TG.add_edge("d", "c", time=2)
TG.add_edge("d", "e", time=2)
TG.add_edge("a", "c", time=2)
TG.add_edge("f", "e", time=3)
TG.add_edge("f", "a", time=3)
TG.add_edge("f", "b", time=3)

TG

Note that the resulting graph object reports a single time step `t=1`,
as it has not yet been [sliced](#slice-temporal-graph).


> **Note:**
> Multigraphs are particularly useful to represent temporal graphs, as it
> allows to store multiple interactions between the same nodes at
> different time steps within a single graph object. This behavior can be
> changed by setting `multigraph=False` when creating the
> [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph) object.

### From a static graph

Static graphs may too carry temporal information in both node- and
edge-level attributes.

Slicing a graph into bins usually result in the same number of edges,
but a higher number of nodes, as they may appear in more than one
snapshot. In the example below, we create a static multigraph in which
both nodes and edges are attributed with the time step in which they are
observed:

In [ ]:
G = nx.DiGraph()

G.add_nodes_from([
    ("a", {"time": 0}),
    ("b", {"time": 0}),
    ("c", {"time": 1}),
    ("d", {"time": 2}),
    ("e", {"time": 3}),
    ("f", {"time": 3}),
])

G.add_edges_from([
    ("a", "b", {"time": 0}),
    ("c", "b", {"time": 1}),
    ("d", "c", {"time": 2}),
    ("d", "e", {"time": 2}),
    ("a", "c", {"time": 2}),
    ("f", "e", {"time": 3}),
    ("f", "a", {"time": 3}),
    ("f", "b", {"time": 3}),
])

G

#### Edge-level time attribute

Converting a static graph with edge-level temporal data to a temporal
graph object:


In [ ]:
TG = tx.from_static(G).slice(attr="time")
tx.draw(TG, layout="kamada_kawai", figsize=(8, 2))

The resulting temporal graph has the same number of edges as the
original graph, but a higher number of nodes. This is expected, as the
same nodes appear in more than one snapshot.


> **Note:**
> By default, [`slice`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.slice) considers `attr` as
> an edge-level attribute, which is usually the case for temporal data.
> This behavior can be changed by setting `attr_level='node'`, as seen below.

#### Node-level time attribute

Converting a static graph with node-level temporal data to a temporal
graph object:


In [ ]:
TG = tx.from_static(G).slice(attr="time", attr_level="node")
tx.draw(TG, layout="kamada_kawai", figsize=(8, 2))

Note that even though the edge $(a, c)$ contains the attribute `time=2`,
considering node-level attributes resulted in it being placed at $t=0$
instead, as the source node $a$ is set to `time=0`:

In [ ]:
G.nodes(data="time")["a"]

> By default, the source node's temporal attribute is used to determine
> the time step of an edge with `attr_level='node'`. This behavior can be
> changed by setting `node_level='target'` instead.

## Slice temporal graph

Let's use the [`slice`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.slice) method to split
the temporal graph we created into a number of snapshots:


In [ ]:
TG = TG.slice(attr="time")
TG

Inspecting the resulting object's properties can be achieved with some
familiar methods:


In [ ]:
print(f"t = {len(TG)} time steps\n"
      f"V = {TG.order()} nodes ({TG.temporal_order()} unique, {TG.total_nodes()} total)\n"
      f"E = {TG.size()} edges ({TG.temporal_size()} unique, {TG.total_edges()} total)")

We may now visualize the resulting snapshots using the
[`draw`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.draw) function:

In [ ]:
tx.draw(TG, layout="kamada_kawai", figsize=(8, 2))


### Specifying number of snapshots

A new object can be created with a specific number of snapshots by
setting the `bins` parameter:

In [ ]:
TG = TG.slice(attr="time", bins=2)
tx.draw(TG, layout="kamada_kawai", figsize=(4, 2), names=True)

> **Hint:**
> Set ``names=True`` to use the [`names`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.names) property as subplot
> titles, instead of their indices. By default, [`slice`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.slice)
> returns the interval of the resulting temporal snapshots as their names.

### Considering quantiles

By default, created bins are composed of non-overlapping edges and might
have uneven order and/or size. To try and balance them using quantiles,
pass `qcut=True` (see
[pandas.qcut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html)
for details):

In [ ]:
TG = TG.slice(attr="time", bins=2, qcut=True)
tx.draw(TG, layout="kamada_kawai", figsize=(4, 2), names=True)

Though not perfectly balanced due to node $a$ appearing multiple times
(in $t={1,2,3}$), the resulting snapshots have a more even number of
edges. Results are expected to vary in a case-by-case basis.


### Ranking nodes or edges

Forcing a number of bins can be achieved by setting `rank_first=True`,
ranking nodes or edges by their order of appearance in the original
graph (see
[pandas.Series.rank](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.rank.html)
for details):


In [ ]:
TG = TG.slice(attr="time", bins=2, rank_first=True)
tx.draw(TG, layout="kamada_kawai", figsize=(4, 2), names=True)

Notice how the intervals in each snapshot title now refer to the edges: $e_0$ to
$e_3$ and $e_4$ to $e_7$. As the `time` attribute is here located at the edge
level, the two snapshots have half of all edges each. In case of node-level times, the number of nodes
in each snapshot would be more evenly distributed.

> **Note:**
> In some cases, it may still not be able to split the graph into the
> number of snapshots specified by `bins` (e.g., due to insufficient
> data), so the maximum possible number is returned instead.

## Save and load data

Temporal graphs may be read from or written to a file using the
following functions:


In [ ]:
tx.write_graph(TG, "temporal_graph.graphml.zip")
TG = tx.read_graph("temporal_graph.graphml.zip")

Supported formats will be automatically detected based on the file
extension. For details on the methods, please refer to their respective
documentation: [`read_graph`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.read_graph) and
[`write_graph`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.write_graph).

> **See also:**
> The [read and write
> documentation](https://networkx.org/documentation/stable/reference/readwrite/index.html)
> from NetworkX for a list of supported graph formats.

## Edge direction

Similar to static NetworkX graphs, edges in a temporal graph can be
easily transformed into directed or undirected by calling the
`to_directed` or `to_undirected`
methods, respectively:

In [ ]:
TG.to_undirected()

In [ ]:
TG.to_directed()

As the methods return new objects, the original graph remains unchanged.
Note that most methods available in the [NetworkX
graphs](https://networkx.org/documentation/stable/reference/classes/graph.html#networkx.Graph)
can be called directly from the [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph)
object as well.

___

# Convert and transform

The package provides a set of functions to convert to different graph
formats and representations. In this context, "converting" refers to
changing the underlying graph object type, e.g.
[igraph](https://igraph.org). while "transforming" refers to changing
the graph representation, e.g., [event-based temporal
graphs](#event-based-temporal-graph).

## Graph formats

Graphs may be converted to a different object type by calling
[`convert`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.convert) with the desired format:

In [ ]:
import networkx_temporal as tx

TG = tx.TemporalGraph(directed=True, multigraph=False)

TG.add_edge("a", "b", time=0)
TG.add_edge("c", "b", time=1)
TG.add_edge("d", "c", time=2)
TG.add_edge("d", "e", time=2)
TG.add_edge("a", "c", time=2)
TG.add_edge("f", "e", time=3)
TG.add_edge("f", "a", time=3)
TG.add_edge("f", "b", time=3)

tx.convert(TG.to_static(), "igraph")

In the example above, the temporal graph `TG` is first flattened using
[`to_static`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.to_static). Otherwise, the function returns a list of graph objects,
one per snapshot, as shown below:

In [ ]:
TG = TG.slice(attr="time")

tx.convert(TG, "igraph")

Support for the following output formats are implemented, here listed
with their respective aliases:

| Format | Parameter (Package) | Parameter (Alias) |
|---|:---:|:---:|
| [Deep Graph Library](https://www.dgl.ai/) | `dgl` | - |
| [graph-tool](https://graph-tool.skewed.de/) | `graph_tool` | `gt` |
| [igraph](https://igraph.org/python/) | `igraph` | `ig` |
| [NetworKit](https://networkit.github.io/) | `networkit` | `nk` |
| [PyTorch Geometric](https://pytorch-geometric.readthedocs.io) | `torch_geometric` | `pyg` |
| [Teneto](https://teneto.readthedocs.io) | `teneto` | - |

## Graph representations

Once a temporal graph is instantiated, the following methods allow
returning static graphs, snapshots events or unified representations.
Due to the way the underlying data is represented, some of these objects
(i.e., those with unique nodes) do not allow dynamic node attributes.

Observe that the total number of nodes $V$ and edges $E$ of the returned
object might differ from the number of temporal nodes $V_T$ and edges
$E_T$, depending on the data and method used:

| Method | Order | Size | Dynamic node attributes | Dynamic edge attributes |
|---|:---:|:---:|:----:|:---:|
| [`to_static`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.to_static) | $V = V_T$ | $E = E_T$ | ❌ | ✅ |
| [`to_snapshots`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.to_snapshots) | $V \ge V_T$ | $E = E_T$ | ✅ | ✅ |
| [`to_events`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.to_events) | $V = V_T$ | $E = E_T$ | ❌ | ❌ |
| [`to_unified`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.to_unified) | $V \ge V_T$ | $E \ge E_T$ | ✅ | ✅ |


### Static graph

Builds a static or flattened graph `G` containing all the edges found at
each time step.

> **Important:**
> Dynamic node attributes in a temporal graph are not preserved in a
> static graph.

**TemporalGraph → G**

In [ ]:
G = TG.to_static()
G

In [ ]:
tx.draw(G, layout="kamada_kawai", suptitle="Static Graph")

**G → TemporalGraph**

In [ ]:
TG = tx.from_static(G).slice(attr="time")
TG

### Snapshot-based temporal graph

A snapshot-based temporal graph `STG` is a sequence of graphs where each
element represents a snapshot of the original temporal graph. It is the
most common representation of temporal graphs.

> **Note:**
> Like the [`slice`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.slice) method, this function
> internally returns views of the original graph data, so no data is
> copied unless specified otherwise, i.e., by passing `as_view=False` to
> the function.

**TemporalGraph → STG**

In [ ]:
STG = TG.to_snapshots()
STG

**STG → TemporalGraph**

In [ ]:
TG = tx.from_snapshots(STG)
TG

### Event-based temporal graph

An event-based temporal graph `ETG` is a sequence of 3- or 4-tuple
edge-based events.

-   **3-tuples** ($u, v, t$), where elements are the source node, target
    node, and time attribute;
-   **4-tuples** ($u, v, t, \epsilon$), where an additional element
    $\epsilon$ is either a positive (`1`) or negative (`-1`) unity
    representing edge addition and deletion events, respectively.

Depending on the temporal graph data, one of these may allow a more
compact representation than the other. The default is to return a
3-tuple sequence (also known as a *stream graph*).

> **Important:**
> Event-based temporal graphs do not currently store node- or edge-level
> attribute data. Moreover, as sequences of events are edge-based, node
> isolates are not preserved.

**TemporalGraph → ETG**

In [ ]:
ETG = TG.to_events()  # stream=True (default)
ETG

In [ ]:
ETG = TG.to_events(stream=False)
ETG

**ETG → TemporalGraph**

In [ ]:
tx.from_events(ETG, directed=True, multigraph=True)

### Unified temporal graph

A unified temporal graph `UTG` is a single graph object that contains
the original temporal data, plus "proxy" nodes (*from each snapshot*)
and edge "couplings" (*linking sequential temporal nodes*). Its
usefulness is restricted to certain types of analysis and visualization,
e.g., based on temporal flows.

**TemporalGraph → UTG**

In [ ]:
UTG = TG.to_unified(add_couplings=True)
UTG

In [ ]:
nodes = sorted(TG.temporal_nodes())

pos = {node: (nodes.index(node.rsplit("_")[0]), -int(node.rsplit("_")[1]))
       for node in UTG.nodes()}

tx.draw(UTG,
        pos=pos,
        figsize=(4, 4),
        connectionstyle="arc3,rad=0.25",
        suptitle="Unified Temporal Graph")

**UTG → TemporalGraph**

In [ ]:
tx.from_unified(UTG)

___

# Common metrics

This section showcases some common metrics available for temporal graphs
and its snapshots, that is, the individual (static) graphs that compose
a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph) object after calling
[`slice`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.slice).


> **Note:**
> Contributions are welcome! If you would like to see a specific metric
> for temporal graphs implemented, please feel free to submit a pull
> request on the package's [GitHub
> repository](https://github.com/nelsonaloysio/networkx-temporal).

## Static graph metrics

The functions and algorithms implemented by NetworkX can be applied
directly on the temporal graph snapshots, either by iterating over them
or by calling the [corresponding
methods](https://networkx.org/documentation/stable/reference/classes/graph.html#methods)
directly.

Such methods are executed on each graph snapshot when called and return
a list of results --- unless in case of specific functions that have
been overriden, maintaining their use, such as
`is_directed`.


### Degree centrality

Methods such as
[degree](https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.degree.html),
[in_degree](https://networkx.org/documentation/stable/reference/classes/generated/networkx.DiGraph.in_degree.html),
and
[out_degree](https://networkx.org/documentation/stable/reference/classes/generated/networkx.DiGraph.out_degree.html)
return a list of degree views per snapshot:

In [ ]:
import networkx_temporal as tx

TG = tx.TemporalGraph(directed=True)

TG.add_edge("a", "b", time=0)
TG.add_edge("c", "b", time=1)
TG.add_edge("d", "c", time=2)
TG.add_edge("d", "e", time=2)
TG.add_edge("a", "c", time=2)
TG.add_edge("f", "e", time=3)
TG.add_edge("f", "a", time=3)
TG.add_edge("f", "b", time=3)

TG = TG.slice(attr="time")

TG.degree()

Alternatively, we may obtain the degree of a specific node in a given
snapshot, e.g., node $a_0$:

In [ ]:
TG[0].degree("a")

### Order and size

Similarly, to obtain the total number of nodes and edges in each
snapshot:

In [ ]:
print("Order:", TG.order())
print("Size:", TG.size())

### Node neighborhoods

The `neighbors` method returns a list of neighbors for each node in each
snapshot:

## Temporal graph metrics

Only a few methods that consider all snapshots are currently available
from [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph) objects. They mostly serve as
wrappers of the available functions in NetworkX, for convenience
purposes.

### Temporal degree centrality

Meanwhile, [`temporal_degree`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_degree) returns a
dictionary containing node degrees across all time steps:

In [ ]:
TG.temporal_degree()

Alternatively, to obtain the degree of a specific node considering all
snapshots, e.g., node $a$:

In [ ]:
TG.temporal_degree("a")

### Temporal order and size

The [`temporal_order`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_order) and
[`temporal_size`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_size) functions return the
total unique nodes and edges:

In [ ]:
print("Temporal nodes:", TG.temporal_order())
print("Temporal edges:", TG.temporal_size())

> **Note:**
> The temporal order and size of a temporal graph match the length of
> [`temporal_nodes`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_nodes) and
> [`temporal_edges`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_edges), i.e., the sets of all
> (**unique**) nodes and edges across all snapshots.

#### Total number of nodes and edges

Obtaining the actual number of nodes and edges across all snapshots,
**with** duplicates:


In [ ]:
print("Total nodes:", TG.total_nodes())  # TG.total_nodes() != TG.temporal_order()
print("Total edges:", TG.total_edges())  # TG.total_edges() == TG.temporal_size()

> **Note:**
> The temporal order and size of a temporal graph match the sum of `nodes`
> and `edges`, i.e., the lists of all (**non-unique**) nodes and edges
> across all snapshots.

### Temporal node neighborhoods

The [`temporal_neighbors`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_neighbors) method returns
a dictionary containing node neighbors in all snapshots:

In [ ]:
TG.temporal_neighbors("c")

___

# Community detection

Community detection is a fundamental task in network analysis. This
simple example demonstrates how a network's temporal dynamics can
benefit the detection of its mesoscale structures.

## Generate graph

As a toy example, let's first use the simplest [Stochastic Block
Model](https://networkx.org/documentation/stable/reference/generated/networkx.generators.community.stochastic_block_model.html)
to generate 4 graph snapshots, in which each of the 5 clusters of 5
nodes each continuously mix together over time:

In [ ]:
snapshots = 4   # Temporal graphs to generate.
clusters = 5    # Number of clusters/communities.
order = 5       # Nodes in each cluster.
intra = .9      # High initial probability of intra-community edges.
inter = .1      # Low initial probability of inter-community edges.
change = .125   # Change in intra- and inter-community edges over time.

# Get probability matrix for each snapshot.
probs = [[[
    (intra if i == j else inter) + (t * change * (-1 if i == j else 1))
    for j in range(clusters)]
    for i in range(clusters)]
    for t in range(snapshots)]

# Create graphs from probabilities.
graphs = {}
for t in range(snapshots):
    graphs[t] = nx.stochastic_block_model(clusters*[order], probs[t], seed=10)
    graphs[t].name = t

# Create temporal graph from snapshots.
TG = tx.from_snapshots(graphs)

Let's plot the graphs, with node colors representing communities and
intra-community edges:

In [ ]:
import matplotlib.pyplot as plt

def get_edge_color(edges: list, node_color: dict):
    return [node_color[u]
            if node_color[u] == node_color[v]
            else "#00000035"
            for u, v in edges]

c = plt.cm.tab10.colors

# Node positions.
pos = nx.circular_layout(TG.to_static())

# Community ground truths.
node_color = [c[i // clusters] for i in range(TG.temporal_order())]

# Colorize intra-community edges.
temporal_opts = {t: {"edge_color": get_edge_color(TG[t].edges(), node_color)}
                 for t in range(len(TG))}

# Plot snapshots with community ground truths.
tx.draw(
    TG,
    pos=pos,
    figsize=(12, 3.5),
    node_color=node_color,
    temporal_opts=temporal_opts,
    connectionstyle="arc3,rad=0.1",
    suptitle="Ground truths")

We see the graphs are generated with the same community structure, but
continuously decreasing assortativity. Let's try and retrieve the ground
truths using a simple community detection algorithm.

## Modularity optimization

The [leidenalg](https://leidenalg.readthedocs.io) [1] package implements
optimization algorithms for community detection that may be applied on
snapshot-based temporal graphs, allowing to better capture their
underlying structure.

> **Attention:**
> Optimizations algorithms may help with descriptive or exploratory
> tasks and post-hoc network analysis, but lack statistical rigor for
> inferential purposes. See [Peixoto
> (2021)](https://skewed.de/tiago/posts/descriptive-inferential/) [2] for
> a discussion.

### On the static graph

Let's start by considering the network as a "flattened" graph, i.e.,
ignoring its temporal information.

We can observe that depending on the initial node community assigments
(e.g., with `seed=0` below),
[modularity](https://leidenalg.readthedocs.io/en/stable/reference.html#modularityvertexpartition) [3]
fails to retrieve the true communities (ground truths) in the
network:

In [ ]:
import leidenalg as la

membership = la.find_partition(
    TG.to_static("igraph"),
    la.ModularityVertexPartition,
    n_iterations=-1,
    seed=0,
)

node_color = [c[m] for m in membership.membership]
edge_color = get_edge_color(TG.to_static().edges(), node_color)

tx.draw(
    TG.to_static(),
    pos=pos,
    figsize=(4, 4),
    node_color=node_color,
    edge_color=edge_color,
    connectionstyle="arc3,rad=0.1",
    suptitle="Communities found by modularity on static graph")

Next, let's try considering the network's temporal information to see if
we can improve the results.

### On each snapshot

Running the same algorithm separately on each of the generated snapshots
retrieves the correct clusters only on the first graph ($t=0$). In
addition, community indices (represented by their colors) are not fixed
over snapshots, which makes understanding their mesoscale dynamics
harder:

In [ ]:
temporal_opts = {}

for t in range(len(TG)):
    membership = la.find_partition(
        TG[t:t+1].to_static("igraph"),
        la.ModularityVertexPartition,
        n_iterations=-1,
        seed=0,
    )
    node_color = [c[m] for m in membership.membership]
    edge_color = get_edge_color(TG[t].edges(), node_color)
    temporal_opts[t] = {"node_color": node_color, "edge_color": edge_color}

tx.draw(
    TG,
    pos=pos,
    figsize=(12, 3.5),
    temporal_opts=temporal_opts,
    connectionstyle="arc3,rad=0.1",
    suptitle="Communities found by modularity on snapshots")

This is mostly due to modularity optimization expecting an assortative
community structure, while our network grows more disassortative over
time. Not only the results of later snapshots are here suboptimal, it is
also particularly hard to understand the network's mesoscale temporal
dynamics.


### On the temporal graph

[Coupling temporal
nodes](https://leidenalg.readthedocs.io/en/stable/multiplex.html#slices-to-layers)
allows the same algorithm to correctly retrieve the ground truths in
this case, while at the same time maintaining community indices
consistent over time, as seen below:

In [ ]:
temporal_opts = {}

temporal_membership, improvement = la.find_partition_temporal(
    TG.to_snapshots("igraph"),
    la.ModularityVertexPartition,
    interslice_weight=1.0,
    n_iterations=-1,
    seed=0,
    vertex_id_attr="_nx_name"
)

for t in range(len(TG)):
    node_color = [c[m] for m in temporal_membership[t]]
    edge_color = get_edge_color(TG[t].edges(), node_color)
    temporal_opts[t] = {"node_color": node_color, "edge_color": edge_color}

tx.draw(
    TG,
    pos=pos,
    figsize=(12, 3.5),
    temporal_opts=temporal_opts,
    connectionstyle="arc3,rad=0.1",
    suptitle="Communities found by modularity on temporal graph")

This method seems particularly useful to track communities over time, as
it allows to maintain the same community indices across snapshots,
potentially contributing to the study of their dynamics. Although very
simple, this example showcases how considering a network's temporal
information can benefit its analysis, as well as help to better
understand and visualize its mesoscale structures.

---

**References**

1. V. A. Traag, L. Waltman, N. J. van Eck (2019). ''From Louvain to Leiden: guaranteeing
   well-connected communities''. Scientific Reports, 9(1), 5233.

2. Tiago. P. Peixoto (2023). ''Descriptive Vs. Inferential Community Detection in Networks:
   Pitfalls, Myths and Half-Truths''. Elements in the Structure and Dynamics of Complex Networks,
   Cambridge University Press.

3. Mark Newman (2018). ''Networks''. Oxford University Press, 2nd ed., pp. 498--514.